In [1]:
import json
import pandas as pd

In [2]:
inpath = '/Users/dongmeichen/OneDrive - lanecouncilofgovernments/VE-RSPM'

In [241]:
scen = pd.read_csv(f'{inpath}/sensitivity_tests/selected_scenario_list.csv')

In [145]:
scen.head()

,B,C,D,E,F,G,I,P,T,V,S
0,1,1,1,1,1,1,1,1,1,1,B1C1D1E1F1G1I1P1T1V1
1,2,1,1,1,1,1,1,1,1,1,B2C1D1E1F1G1I1P1T1V1
2,1,2,1,1,1,1,1,1,1,1,B1C2D1E1F1G1I1P1T1V1
3,2,2,1,1,1,1,1,1,1,1,B2C2D1E1F1G1I1P1T1V1
4,1,3,1,1,1,1,1,1,1,1,B1C3D1E1F1G1I1P1T1V1


In [242]:
scen = scen.rename(columns={'S': 'Scenario'})
#scen.rename(columns={'S': 'Scenario'}, inplace=True)
scen.head()

,B,C,D,E,F,G,I,P,T,V,Scenario
0,1,1,1,1,1,1,1,1,1,1,B1C1D1E1F1G1I1P1T1V1
1,2,1,1,1,1,1,1,1,1,1,B2C1D1E1F1G1I1P1T1V1
2,1,2,1,1,1,1,1,1,1,1,B1C2D1E1F1G1I1P1T1V1
3,2,2,1,1,1,1,1,1,1,1,B2C2D1E1F1G1I1P1T1V1
4,1,3,1,1,1,1,1,1,1,1,B1C3D1E1F1G1I1P1T1V1


In [243]:
meas = pd.read_csv(f'{inpath}/output/Measures_Sensitivity_Output_5.csv')
meas.head()

,Measure,Units,Description,Value2010,B1C1D1E1F1G1I1P1T1V1,B2C1D1E1F1G1I1P1T1V1,B1C2D1E1F1G1I1P1T1V1,B2C2D1E1F1G1I1P1T1V1,B1C3D1E1F1G1I1P1T1V1,B2C3D1E1F1G1I1P1T1V1,B1C1D2E1F1G1I1P1T1V1,B2C1D2E1F1G1I1P1T1V1,B1C2D2E1F1G1I1P1T1V1,B2C2D2E1F1G1I1P1T1V1
0,HhTotDailyOthParkingCost,USD per day,Total daily non-work parking expenditures by h...,28241.275835,20961.508423,20961.508423,20961.508423,20961.508423,20961.508423,20961.508423,20961.508423,20961.508423,20961.508423,20961.508423
1,HhTotDailyWkrParkingCost,USD per day,Total daily work parking expenditures by house...,37225.367097,46029.338718,46029.338718,46029.338718,46029.338718,46029.338718,46029.338718,46029.338718,46029.338718,46029.338718,46029.338718
2,HhVehTravCostShare,Proportion,Household vehicle Own & out-of-pocket costs sh...,0.171750,0.044645,0.044276,0.044850,0.044480,0.045082,0.044712,0.044531,0.044164,0.044737,0.044370
3,HhVehTravCostShareLowInc,Proportion,Low income (0to20K 2010$) household vehicle Ow...,0.312967,0.339122,0.339121,0.343208,0.343208,0.347504,0.347503,0.339122,0.339121,0.343208,0.343207
4,MareaCarSvcAutoDvmtProp,Proportion,Average proportion car service vehicle DVMT in...,1.000000,0.981836,0.981834,0.982353,0.982349,0.981308,0.981306,0.981838,0.981832,0.982352,0.982349


In [102]:
for x in scen.iterrows():
    print(x)
    break

(0, B                              2
C                              3
D                              3
E                              1
F                              2
G                              2
I                              3
P                              2
T                              1
V                              1
Scenario    B2C3D3E1F2G2I3P2T1V1
Name: 0, dtype: object)


In [8]:
def rows_to_dicts(df, data_info=['measure', 'scenario']):
    out = []
    fields = list(df)
    if data_info == 'measure':
        out = [{field: val for field, val in zip(fields, values)} 
           for _, values in df.iterrows()]
    elif data_info == 'scenario':
        out = [{field: [val] for field, val in zip(fields, values)} 
           for _, values in df.iterrows()]
    return out

In [244]:
scen = scen.astype(str)

In [245]:
fields = list(scen)
fields.pop(-1)

'Scenario'

In [246]:
fields

['B', 'C', 'D', 'E', 'F', 'G', 'I', 'P', 'T', 'V']

In [247]:
column_names = ['Scenario']
for col in fields:
    column_names.append(col)

In [248]:
column_names

['Scenario', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'P', 'T', 'V']

In [249]:
scen = scen.reindex(columns=column_names)

In [250]:
scen.head()

,Scenario,B,C,D,E,F,G,I,P,T,V
0,B1C1D1E1F1G1I1P1T1V1,1,1,1,1,1,1,1,1,1,1
1,B2C1D1E1F1G1I1P1T1V1,2,1,1,1,1,1,1,1,1,1
2,B1C2D1E1F1G1I1P1T1V1,1,2,1,1,1,1,1,1,1,1
3,B2C2D1E1F1G1I1P1T1V1,2,2,1,1,1,1,1,1,1,1
4,B1C3D1E1F1G1I1P1T1V1,1,3,1,1,1,1,1,1,1,1


In [251]:
output = rows_to_dicts(scen, data_info='measure')

In [252]:
print(len(output))
len(scen.Scenario.unique())

10


10

In [24]:
co2_vals = [x for x in meas['Measure'] if 'CO2' in x]
co2_vals

['UrbanBusCO2e',
 'UrbanBusCO2eRate',
 'UrbanComSvcCO2e',
 'UrbanHhCO2e',
 'UrbanHvyTrkAveCO2eRate',
 'UrbanHvyTrkCO2e',
 'UrbanLdvCO2e',
 'UrbanLdvCO2eRate',
 'UrbanRailCO2e',
 'UrbanVanCO2e']

In [25]:
len(co2_vals)

10

In [26]:
def vals_from_scenario(scenario, meas_df, subset=None):
    subset = meas_df.Measure.unique() if subset is None else subset
    try:
        out = {field: meas_df.loc[meas_df.Measure == field, scenario].values[0]
              for field in subset}
        return out
    except KeyError:
        return {}
    except:
        raise

In [29]:
def append_scenario_vals(outputs, meas_df, subset=None):
    for d in outputs:
        scenario = d['Scenario']
        scenario_vals = vals_from_scenario(scenario, meas_df, subset)
        d.update(scenario_vals)
    return outputs

In [253]:
test = append_scenario_vals(output, meas, co2_vals)

In [254]:
test[-1]

{'Scenario': 'B2C2D2E1F1G1I1P1T1V1',
 'B': '2',
 'C': '2',
 'D': '2',
 'E': '1',
 'F': '1',
 'G': '1',
 'I': '1',
 'P': '1',
 'T': '1',
 'V': '1',
 'UrbanBusCO2e': 6354.245369851839,
 'UrbanBusCO2eRate': 1209.5679405526598,
 'UrbanComSvcCO2e': 33065.147048568,
 'UrbanHhCO2e': 442204.77486043295,
 'UrbanHvyTrkAveCO2eRate': 855.3373801353861,
 'UrbanHvyTrkCO2e': 632777.675925398,
 'UrbanLdvCO2e': 482502.060671834,
 'UrbanLdvCO2eRate': 0.00016008580337814398,
 'UrbanRailCO2e': 0.0,
 'UrbanVanCO2e': 7232.13876283271}

In [255]:
with open(f'{inpath}/VEScenarioViewer/test_out.json', 'w') as f:
    json.dump(test, f)

In [38]:
#!open ./test_out.json

In [39]:
#!rm ./test_out.json

In [33]:
# organize measure notes
meas_notes = meas[["Measure", "Units", "Description"]]

In [34]:
meas_notes.head()

,Measure,Units,Description
0,HhTotDailyOthParkingCost,USD per day,Total daily non-work parking expenditures by h...
1,HhTotDailyWkrParkingCost,USD per day,Total daily work parking expenditures by house...
2,HhVehTravCostShare,Proportion,Household vehicle Own & out-of-pocket costs sh...
3,HhVehTravCostShareLowInc,Proportion,Low income (0to20K 2010$) household vehicle Ow...
4,MareaCarSvcAutoDvmtProp,Proportion,Average proportion car service vehicle DVMT in...


In [35]:
# NAME, LABEL, DESCRIPTION, INSTRUCTIONS, METRIC, UNIT, COLUMN
meas_notes = meas_notes.rename(columns={'Measure': 'NAME', 'Units': 'UNIT', 'Description':'DESCRIPTION'})

In [36]:
meas_notes.head()

,NAME,UNIT,DESCRIPTION
0,HhTotDailyOthParkingCost,USD per day,Total daily non-work parking expenditures by h...
1,HhTotDailyWkrParkingCost,USD per day,Total daily work parking expenditures by house...
2,HhVehTravCostShare,Proportion,Household vehicle Own & out-of-pocket costs sh...
3,HhVehTravCostShareLowInc,Proportion,Low income (0to20K 2010$) household vehicle Ow...
4,MareaCarSvcAutoDvmtProp,Proportion,Average proportion car service vehicle DVMT in...


In [37]:
meas_notes.loc[:,'LABEL'] = meas_notes.loc[:,'NAME']
meas_notes.loc[:,'COLUMN']= meas_notes.loc[:,'NAME']

In [38]:
meas_notes.head()

,NAME,UNIT,DESCRIPTION,LABEL,COLUMN
0,HhTotDailyOthParkingCost,USD per day,Total daily non-work parking expenditures by h...,HhTotDailyOthParkingCost,HhTotDailyOthParkingCost
1,HhTotDailyWkrParkingCost,USD per day,Total daily work parking expenditures by house...,HhTotDailyWkrParkingCost,HhTotDailyWkrParkingCost
2,HhVehTravCostShare,Proportion,Household vehicle Own & out-of-pocket costs sh...,HhVehTravCostShare,HhVehTravCostShare
3,HhVehTravCostShareLowInc,Proportion,Low income (0to20K 2010$) household vehicle Ow...,HhVehTravCostShareLowInc,HhVehTravCostShareLowInc
4,MareaCarSvcAutoDvmtProp,Proportion,Average proportion car service vehicle DVMT in...,MareaCarSvcAutoDvmtProp,MareaCarSvcAutoDvmtProp


In [39]:
meas_notes.loc[:,'INSTRUCTIONS'] = meas_notes.loc[:,'DESCRIPTION']

In [40]:
meas_notes.loc[:,'METRIC'] = 'Average'

In [41]:
meas_notes = meas_notes[['NAME', 'LABEL', 'DESCRIPTION', 'INSTRUCTIONS', 'METRIC', 'UNIT', 'COLUMN']]

In [42]:
meas_notes.head()

,NAME,LABEL,DESCRIPTION,INSTRUCTIONS,METRIC,UNIT,COLUMN
0,HhTotDailyOthParkingCost,HhTotDailyOthParkingCost,Total daily non-work parking expenditures by h...,Total daily non-work parking expenditures by h...,Average,USD per day,HhTotDailyOthParkingCost
1,HhTotDailyWkrParkingCost,HhTotDailyWkrParkingCost,Total daily work parking expenditures by house...,Total daily work parking expenditures by house...,Average,USD per day,HhTotDailyWkrParkingCost
2,HhVehTravCostShare,HhVehTravCostShare,Household vehicle Own & out-of-pocket costs sh...,Household vehicle Own & out-of-pocket costs sh...,Average,Proportion,HhVehTravCostShare
3,HhVehTravCostShareLowInc,HhVehTravCostShareLowInc,Low income (0to20K 2010$) household vehicle Ow...,Low income (0to20K 2010$) household vehicle Ow...,Average,Proportion,HhVehTravCostShareLowInc
4,MareaCarSvcAutoDvmtProp,MareaCarSvcAutoDvmtProp,Average proportion car service vehicle DVMT in...,Average proportion car service vehicle DVMT in...,Average,Proportion,MareaCarSvcAutoDvmtProp


In [43]:
meas_notes_subset = meas_notes.loc[meas_notes['NAME'].isin(co2_vals)]

In [51]:
output2 = rows_to_dicts(meas_notes_subset,data_info='measure')

In [53]:
with open(f'{inpath}/VEScenarioViewer/test_out2.json', 'w') as f:
    json.dump(output2, f)

In [224]:
# organize category and levels
cat_levels = pd.read_csv(f'{inpath}/sensitivity_tests/sensitivity_scenarios.csv')

In [225]:
cat_levels.head()

,strategy_label,strategy_description,strategy_level,category_name,category_label,category_description,category_instructions,file,geo,policy_name,policy_label,policy_description
0,Community Design,Local policies to enable shorter trips and tra...,1,B,Bicycles,"Network improvements, incentives, and technolo...",The diversion of single-occupant vehicle trave...,azone_prop_sov_dvmt_diverted.csv,azone,1,Base,Baseline bicyling percentage of SOV tours less...
1,Community Design,Local policies to enable shorter trips and tra...,2,B,Bicycles,"Network improvements, incentives, and technolo...",The diversion of single-occupant vehicle trave...,azone_prop_sov_dvmt_diverted.csv,azone,2,Increased,Increased bicyling percentage of SOV tours les...
2,Community Design,Local policies to enable shorter trips and tra...,1,P,Parking,Extent of and charges for priced parking,The extent of paid parking and its price,bzone_parking.csv,bzone,1,Base,Baseline average daily cost for long-term park...
3,Community Design,Local policies to enable shorter trips and tra...,2,P,Parking,Extent of and charges for priced parking,The extent of paid parking and its price,bzone_parking.csv,bzone,2,Increased by 50%,Increased average daily cost for long-term par...
4,Community Design,Local policies to enable shorter trips and tra...,3,P,Parking,Extent of and charges for priced parking,The extent of paid parking and its price,bzone_parking.csv,bzone,3,Double,Double average daily cost for long-term parkin...


In [226]:
list(cat_levels.columns)

['strategy_label',
 'strategy_description',
 'strategy_level',
 'category_name',
 'category_label',
 'category_description',
 'category_instructions',
 'file',
 'geo',
 'policy_name',
 'policy_label',
 'policy_description']

In [227]:
# NAME, LABEL, DESCRIPTION, INSTRUCTIONS, LEVELS (NAME, LABEL, DESCRIPTION)
levels = cat_levels[['policy_name', 'policy_label', 'policy_description']]

In [228]:
levels = levels.rename(columns={'policy_name': 'NAME', 'policy_label': 'LABEL', 'policy_description':'DESCRIPTION'})

In [229]:
levels.head()

,NAME,LABEL,DESCRIPTION
0,1,Base,Baseline bicyling percentage of SOV tours less...
1,2,Increased,Increased bicyling percentage of SOV tours les...
2,1,Base,Baseline average daily cost for long-term park...
3,2,Increased by 50%,Increased average daily cost for long-term par...
4,3,Double,Double average daily cost for long-term parkin...


In [230]:
level_output = rows_to_dicts(levels, data_info='scenario')
level_output[0]

{'NAME': [1],
 'LABEL': ['Base'],
 'DESCRIPTION': ['Baseline bicyling percentage of SOV tours less than 20 miles (19%)']}

In [231]:
type(level_output[0])

dict

In [232]:
categories = cat_levels[['category_name','category_label','category_description','category_instructions']]
categories = categories.rename(columns={'category_name': 'NAME', 
                                        'category_label': 'LABEL', 
                                        'category_description':'DESCRIPTION',
                                        'category_instructions': 'INSTRUCTIONS'})

In [233]:
category_output = rows_to_dicts(categories, data_info='scenario')
category_output[0]

{'NAME': ['B'],
 'LABEL': ['Bicycles'],
 'DESCRIPTION': ['Network improvements, incentives, and technologies that encourage bicycling and other light-weight vehicle travel'],
 'INSTRUCTIONS': ['The diversion of single-occupant vehicle travel to bicycles, electric bicycles and other light-weight vehicles']}

In [207]:
def list_levels_in_cat(cat, cat_levels_df):
    df = cat_levels_df.loc[cat_levels_df['category_name'] == cat]
    df = df[['policy_name', 'policy_label', 'policy_description']]
    df = df.rename(columns={'policy_name': 'NAME',
                            'policy_label': 'LABEL',
                            'policy_description':'DESCRIPTION'})
    out = rows_to_dicts(df, data_info='scenario')
    return out

In [208]:
def list_cateory_info(cat, cat_levels_df):
    df = cat_levels_df.loc[cat_levels_df['category_name'] == cat]
    df = df[['category_name','category_label','category_description','category_instructions']].drop_duplicates()
    df = df.rename(columns={'category_name': 'NAME', 
                            'category_label': 'LABEL', 
                            'category_description':'DESCRIPTION',
                            'category_instructions': 'INSTRUCTIONS'})
    out = rows_to_dicts(df, data_info='scenario')
    return out    

In [234]:
cat_levels.policy_name = cat_levels.policy_name.astype(str)

In [235]:
list_levels_in_cat('B', cat_levels)

[{'NAME': ['1'],
  'LABEL': ['Base'],
  'DESCRIPTION': ['Baseline bicyling percentage of SOV tours less than 20 miles (19%)']},
 {'NAME': ['2'],
  'LABEL': ['Increased'],
  'DESCRIPTION': ['Increased bicyling percentage of SOV tours less than 20 miles']}]

In [236]:
cat_levels[['category_name','category_label','category_description','category_instructions']].drop_duplicates().head()

,category_name,category_label,category_description,category_instructions
0,B,Bicycles,"Network improvements, incentives, and technolo...",The diversion of single-occupant vehicle trave...
2,P,Parking,Extent of and charges for priced parking,The extent of paid parking and its price
5,T,Transit,Level of public transit service,The extent and frequency of transit service
8,D,Demand Management,Programs to encourage less private vehicle travel,Programs and incentives which encourage people...
11,E,Driving Efficiency,Driving efficiency by increasing implementatio...,DVMT affected by operations for different road...


In [237]:
list_cateory_info('B', cat_levels)

[{'NAME': ['B'],
  'LABEL': ['Bicycles'],
  'DESCRIPTION': ['Network improvements, incentives, and technologies that encourage bicycling and other light-weight vehicle travel'],
  'INSTRUCTIONS': ['The diversion of single-occupant vehicle travel to bicycles, electric bicycles and other light-weight vehicles']}]

In [213]:
def add_nested_dicts(cat_dict, level_list):
    cat_dict['LEVELS'] = level_list
    return cat_dict

In [214]:
def collect_scenario_info(cat_levels_df):
    out = []
    for cat in list(cat_levels_df.category_name.unique()):
        list1 = list_cateory_info(cat, cat_levels_df)
        list2 = list_levels_in_cat(cat, cat_levels_df)
        out.append(add_nested_dicts(list1[0], list2))
    return out

In [238]:
collect_scenario_info(cat_levels)[0]

{'NAME': ['B'],
 'LABEL': ['Bicycles'],
 'DESCRIPTION': ['Network improvements, incentives, and technologies that encourage bicycling and other light-weight vehicle travel'],
 'INSTRUCTIONS': ['The diversion of single-occupant vehicle travel to bicycles, electric bicycles and other light-weight vehicles'],
 'LEVELS': [{'NAME': ['1'],
   'LABEL': ['Base'],
   'DESCRIPTION': ['Baseline bicyling percentage of SOV tours less than 20 miles (19%)']},
  {'NAME': ['2'],
   'LABEL': ['Increased'],
   'DESCRIPTION': ['Increased bicyling percentage of SOV tours less than 20 miles']}]}

In [239]:
output3 = collect_scenario_info(cat_levels)

In [240]:
with open(f'{inpath}/VEScenarioViewer/test_out3.json', 'w') as f:
    json.dump(output3, f)

In [257]:
# organize average values for output measures
scen_fields = set(list(meas.columns)).intersection(set(list(scen.loc[:,'Scenario'])))

In [258]:
meas['Avg'] = meas.loc[:, scen_fields].mean(axis=1)

In [259]:
meas_s = meas.loc[meas.Measure.isin(co2_vals),:][['Measure', 'Avg']]

In [260]:
meas_s.head()

,Measure,Avg
18,UrbanBusCO2e,6354.245370
19,UrbanBusCO2eRate,1209.567941
22,UrbanComSvcCO2e,33416.530242
26,UrbanHhCO2e,447079.006752
64,UrbanHvyTrkAveCO2eRate,855.449000


In [261]:
output4 = {m: a for m, a in zip(meas_s.Measure, meas_s.Avg)}

In [262]:
with open(f'{inpath}/VEScenarioViewer/test_out4.json', 'w') as f:
    json.dump(output4, f)

In [263]:
output4

{'UrbanBusCO2e': 6354.245369851838,
 'UrbanBusCO2eRate': 1209.56794055266,
 'UrbanComSvcCO2e': 33416.53024216243,
 'UrbanHhCO2e': 447079.00675226597,
 'UrbanHvyTrkAveCO2eRate': 855.4490001201823,
 'UrbanHvyTrkCO2e': 632860.2522703665,
 'UrbanLdvCO2e': 487727.67575726146,
 'UrbanLdvCO2eRate': 0.00016009293958940958,
 'UrbanRailCO2e': 0.0,
 'UrbanVanCO2e': 7232.138762832708}